In [1]:
import pymc3 as pm
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from arviz.utils import Numba
Numba.disable_numba()
Numba.numba_flag

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


False

In [ ]:
# AB테스트를 하는 과정에서 손실이 일어날 수 있는데 손실을 최적화하는 방법은?

# 활용exploitation : 자신이 알고 있는 최적의 행동을 함으로써 이익을 높이는 것
# 탐색exploration : 지식을 늘리기 위해 새로운 행동을 시도

# ==>탐색과 활용의 틸레마 exploration-exploitation dilemma

# 다중 슬롯머신 문제 multi-armed bandit problem : 탐색과 활용의 딜레마를 안고 있는 문제를 다루는 전형적인 방법
# 여러개의 슬롯머신이 있을 때 어느 슬롯머신에 배팅할 것인가? 
# 여기서는 확률적 슬롯머신 문제를 다룸. 솔루션을 선택했을 때의 보상이 어떤 확률 분포에 따라 주어진다고 가정하며, 누적 보상을 최대화 하고자 함.
# 에이전트agent : 최적의 솔루션을 찾아내는 주체. 총 T 횟수만큼 다양한 솔루션을 시도.
# 환경environment : 에이전트와 대치하는 대상 전체
# T: 예산. 횟수 상한선. 1 <= t <= T 는 '시각'이라고 부름
# x(t) : t번째 횟수에 선택한 솔루션
# r(t) : t번째 횟수에 얻은 보상 (웹사이트 최적화의 경우 클릭 유무)
# R(t) : 누적 보상. 이를 최대화하는 전략, 즉 방책을 구하는 것이 확률적 슬롯머신 문제

# 슬롯머신 알고리즘의 성능은 많은 횟수를 시뮬레이션한 결과를 바탕으로 누적 보상 기댓값을 계산에 평가함. 누적 보상 기닷값이 이상적인 방책에 얼마나 가까운지를 평가.
# 이상적인 방책이란 항상 기댓값이 최고인 솔루션을 계속 선택하는 가상의 방책
# 후회regret : 이상적인 방책과 누적 보상의 차이. 


# epsilon-greedy 알고리즘: 슬롯머신 알고리즘 중 가장 간단한 알고리즘.
# 어떤 작은 확률 e으로 탐색, 1 - e 로 활용 행동을 취하는 것. 
# 탐색 행동을 선택한 경우에는 솔루션을 무작위로 선택
# 활용 행동ㅇ르 선택한 경우에는 그 시점에서 얻어진 보상의 표본 평균이 최대인 솔루션 x*(t)를 선택


In [2]:
# epsilon-greedy 알고리즘 구현하기:
# 보상 r : 성공확률에 따라 0 또는 1의 값
# 각 솔루션의 성공 확률: theta1 = 0.1, theta2 = 0.2, theta3 = 0.3, theta4 = 0.3
# 분제의 최적의 솔루션 : x* = x4

import numpy as np
np.random.seed(0)
n_arms = 4  #솔루션 수 K = 4

class Env(object):
    thetas = [0.1, 0.1, 0.2, 0.3]
    
    def react(arm):
        return 1 if np.random.random() < Env.thetas[arm] else 0
    
    def opt():  #전지전능한 에이전트 외에는 이 메서드 호출할 수 없음
        return np.argmax(Env.thetas)

In [ ]:
class EpsilonGreedyAgent(object):
    
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_amrs)
        
    def get_arm(self):
        if np.random.random() < self.epsilon:
            arm = np.random.randint(n_arms)
        else:
            arm = np.argmax(self.values)
        return arm
    
    def sample(self, arm, reward):
        
        